<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Diode 04B - Rectifiers (Schottky Version)

This project deals with one of the most typical diode application: Signal rectification.

There are two versions of this document, version **"A"** that uses standard **1N41448** diodes and version **"B"** uses **BAT42** schottky diodes. You are now reading the first **"A"** version. 

Rectifier measurements require the use of bipolar (negative and positive) voltages, but the **SLab** system is unipolar (only positive) with its generated voltages restricted to a range between **GND** and **Vdd**. There are two main methods to generate bipolar voltages from unipolar sources and both require to use two voltage sources connected to circuit. 

* The **Bridge** méthod, associated to the other document, uses the full range of voltages for each source an can get an input range that doubles the **Vdd** range giving circuit voltages between $-Vdd$ and $Vdd$. This is the method used in the **"A"** version of this document.


* The **Virtual Ground** method, associated to this document, sets one voltage source at a constant $Vdd/2$ value and varies the other source from **GND** to **Vdd**. That gives circuit voltage ranges between $-Vdd/s$ and $Vdd/2$. This is the method used in the **"B"** version of this document. As the voltage range is half of the one on the **"A"** version, the **"B"** version uses **Schottky** diodes that feature a lower voltage drop and thus are easier to use in a reduced voltage range system.

Version 1.0 (13/3/2019)  
License information is at the end of the document

---
**Bill Of Materials (BOM):**

* Diodes: 4x **BAT42** Schottky diodes  
* Capacitors: 1x $10 \mu F$ capacitor
* Resistors: 2x $1k\Omega$, 2x $10k\Omega$, 1x $100k\Omega$  
* Two additional resistors to be calculated  

---

## Introduction

One of the main uses of the diodes is **signal rectification**. This very useful in two important electronic domains: power supplies and signal demodulation. 

Respect to power supplies, most electronic devices power on DC voltages, but we get on the wall socket an AC voltage. So, most electronic devices that are connected to mains need to rectify and filter the AC voltage to obtain a DC voltage to power its electronics. 

Respect to demodulation, in the case of over the air signal transmission, like AM broadcasts, the audio signal is transmitted by modulating an AM radio frequency carrier. There are other systems that use amplitude modulation like FSK data transmissions and most Infrared remote control units.

In order to remove the carrier from the amplitude modulated signal, rectification can also be used. 
In this document we will analyze the basic rectifier and we will use it do rectify and filter a sine wave. We will also use it to demodulate and amplitude modulated signal.

But, before we start working with the rectifier, we will need to deal to a voltage range problem associated to the SLab system. This will also serve us to introduce the bridge excitation of circuits.

## The basic rectifier

The following circuit shows the basic rectifier circuit. This circuit takes an input voltage generated from a $V_S$ voltage source. Rectifies it using a **D diode** and provides the rectified voltage to a $R_L$ load.

![Basic Rectifier](images/D_04/half_wave.png)

In order to analyze the circuit we will use the large signal constant voltage diode model that assumes that the diode voltage is $V_\gamma$ when conducts. 

$\qquad V_d = V_\gamma \qquad I_d \geq 0 \qquad$ ON State  

$\qquad I_d = 0 \qquad V_d \leq 0 \qquad$ OFF State  

The diode can only be in two states: **ON** or **OFF**. In **ON** state $V_d = V_\gamma$. We can obtain $V_L$ as function of $V_S$ and $V_\gamma$ in this case. As, the circuit in **ON** state is linear, $V_L(V_S)$ will be a linear response.

The model requires that $I_d \geq 0$, so we can also calculate $I_d$ and check the range of input voltages that make true the inequation.
If you perform the analysis you should get:

$$V_L = V_S - V_\gamma \qquad \qquad V_S \geq V_\gamma$$

---

**CALCULATION TASK**  
Solve the circuit for the **ON** state and check that the solution and the conditions you get are the above ones.  

---

In **OFF state**, current in the diode is zero and we require that $V_d \leq V_\gamma$. If you solve the circuit for this case you should get the following solution and condition for $V_L$.

$$V_L = 0 \qquad \qquad V_S \leq V_\gamma$$

As the diode can only be in two states, the condition for $V_S$ in **OFF** state need to be complementary to the one for the **ON** state. This is not mathematically exact as $V_S = V_\gamma$ is compatible and give the same solution to both states.

![Basic Rectifier Response](images/D_04/half_wave_curve.png)

The solution for the **OFF** state for every $V_S$ value below $V_\gamma$ whereas the solution for the **ON** state is valid for every value of $V_S$ over $V_\gamma$. The solution $V_L(V_S)$ for each state is a straight line and, as both state solutions are true for $V_S = V_\gamma$ the two lines crosses just at this point.

---

**CALCULATION TASK**  
Solve the circuit for the **OFF** state and check that the solution and the conditions you get are the expected ones.

Draw in a graph $V_L(V_S)$ assuming $V_\gamma=0.3 V$ for $V_S$ voltages between  $3 V$ and $+3 V$.   
It should be similar to the above figure.

---


The curve you have just drawn is the **Input** to **Output** DC response of the rectifier circuit.  
Now we will measure it to test if we get same values obtained in the previous calculations.

---

![Practical Icon](images/pt.png)

---



First, we will import the main SLab module and the DC module.  
Afterwards we will connect with the board.  

In [ ]:
# Import the main SLab module
import slab

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

It is always a good idea to check the board operation.  
The following cell checks the calibration for the **DAC** channels and the first four **ADC** channels.  

In [ ]:
# Check the calibration
slab.checkCalibration(pause=False,na=4,nm=10)

We will implement the previous shown rectifier circuit using a **BAT42** Schottky diode. We will also use a $1 k\Omega$ resistor for $R_L$. The following schematic shows all the connections on the test circuit. Observe that the ground terminal is not directly connected to any of the components you will mount on the breadboard. 

![Basic Rectifier Schematis](images/D_04/half_wave_sch2.png)

We don't use the **ground** terminal because the **SLab** system is unipolar and can only generate or measure positive voltages. In order to work with negative voltages we need to use a positive voltage as our **virtual ground**. In particular we will use **DAC2** for that set for a $1.6 V$ value halfway in the supply $3.3V$ range.

After mounting the components we use the following code cell to set **DAC2** voltage and generate a sweep of the **DAC1** voltage. Then we will calculate $V_i$ and $V_o$ taking into account the **virtual ground** we are using.

$\qquad V_i = V_{ADC3} - V_{ADC2}$

$\qquad V_o = V_{ADC1} - V_{ADC2}$

Execute the following code cell to perfrom the measurements.

In [ ]:
# Obtain the DC Voltage I/O response of the rectifier

slab.setVoltage(2,1.6)            # Set virtual ground on DAC2
data=slab.dcSweep(1,0.0,3.2,0.1)  # Sweep DAC1 voltage

vi = data[3] - data[2]   # Compute Vi
vo = data[1] - data[2]   # Compute Vo

# Plot Vo against Vi
slab.plot11(vi,vo,'Basic Rectifier DC Response','Vi[V]','Vo[V]')

You can now compare obtained curve with the one you have drawn from the theoretical calculations.

The two curves should match quite well, perhaps the $V_S$ value that defines the transition from the **OFF** to the **ON** state is not exactly the same but it should be near.

Now that we have obtained the DC curve, we can test the circuit against a sine wave. As $1.6V$ is our virtual ground, we will generate a wave that is centered on this voltage:

$$V_{DAC1}=1.6V + 1.2V \cdot sin(\omega t)$$

So the voltage at the input of the circuit will be:

$$V_i = V_{DAC1}-V_{DAC2} = 1.2V \cdot sin(\omega t)$$

The following code configures the sine waves with **100** data points and **50 Hz** frequency.  
Then we configure the transient capture to record five full waves and we perform a measurement and store all the four **ADC** channels.

In [ ]:
# Transient measurement on the basic rectifier

slab.waveSine(0.4,2.8,100)              # Generate a sinewave for DAC1 
slab.setWaveFrequency(50)               # Set the wave frequency to 50Hz
slab.tranStore(500,4)                   # Store 5 full waves for all 4 channels

# Perform the transient measurement using both DACs and returning the measurement data
t,a1,a2,a3,a4 = slab.wavePlot(returnData=True)

In order to obtain the input and output voltages we will need to substract the virtual ground voltage:

In [ ]:
vi = a3 - a2  # Calculate Vi
vo = a1 - a2  # Calculate Vo

# Plot Vi and Vo against time
slab.plot1n(t,[vi,vo],"Simple Rectifier Response","time (s)","Voltage (V)",["Vi","Vo"])

You should see that only the positive part of the input signal passes the diode and reaches the load, for the negative part of the input signal, the load voltage is zero.

You can also see the voltage on the diode:

In [ ]:
vd=vi-vo  # Calculate diode voltage

slab.plot11(t,vd,"Diode Voltage","time (s)","Vd (V)") # Plot it

You should get a voltage drop arround $0.3 V$ awhen the diode conducts in the **ON** state, and the full input voltage when it blocks in the **OFF** state.

## Filtering the output

The previous circuit rectifies the input signal as it only lets pass the positive part of it, but the output voltage is not DC as it changes with time. If we want to use the rectifier to generate a DC voltage we must filter the output signal.

The easiest way to filter the signal is to add a capacitor in parallel with the load.

![Filter](images/D_04/filter.png)

The capacitor, as it blocks the DC current does not change the DC curve of the circuit, but it has a great impact on its transient behavior. We could try to analyze this circuit before measuring it, but, this time, having the response at hand makes it easier to explain its behavior. 

The schematic is similar to the previous one. We just add a capacitor.

![Filtered Rectifier](images/D_04/filtered_sch2.png)

----

**BUILD TASK**  
Add the capactitor to the circuit  

---

The following code will repeat the previous measurements for the new circuit. Observe that we don't need to upload the sine waveform as it is still inside the board.

In [ ]:
# Measurement for the filtered basic rectifier

# Perform the transient measurement using both DACs and returning the measurement data
t,a1,a2,a3,a4 = slab.wavePlot(returnData=True)

Like in the previous case, we need to perform some calculations to obtain the input and output voltages for the circuit.

In [ ]:
vi = a3 - a2  # Calculate Vi
vo = a1 - a2  # Calculate Vo

# Plot Vi and Vo against time
slab.plot1n(t,[vi,vo],"Simple Filtered Rectifier Response","time (s)","Voltage (V)",["Vi","Vo"])

The input should be a sine wave between  $-1.2 V$ and $1.2 V$ and the output should be have a sawtooth shape. It is normal for the input to have some deformation at the higher voltages.

The SLab system doesn't use continuous waves, waves are only generated during the measurements. That is good in this case because we can learn how the circuit operates from the beginning. The following lines describe how the circuit works: 

* Before the measurement starts, the capacitor is discharged so its voltage is zero.


* The $V_i$ input sine wave starts at zero and goes up.


* No change is seen on the output voltage $V_o$ until the input voltage reaches the threshold $V_\gamma$ voltage of the diode. Diode is in **OFF** state until this time. 


* From this point, until $V_i$ reaches its maximum, diode is in **ON** state and $V_o$ is one diode drop $V_\gamma$ below $V_i$.


* After $V_i$ reaches the maximum, it starts to drop. The capacitor conserves its voltage so the voltage on the diode lowers and it goes to the **OFF** state.


* Voltage on the capacitor is not constant, however, because it is discharged through the resistor in a typical exponential fashion.


* Diode is kept in the **OFF** state until, on the next cycle, the $V_i$ to $V_o$ difference reaches $V_\gamma$ and the diode returns to the **ON** state.


* From this point onwards, the response of the circuit repeats so it is periodic.

We can see that the ouput voltage is not a DC constant but has a ripple sawtooth signal. We can roughly calculate the ripple of the voltage from the exponential decay of the capacitor voltage.

When the diode is **OFF**, the capacitor and the load resistors are isolated from the rest of the circuit, so they will feature an exponential discharge:

$$V_L(t)=V_{L0}\cdot e^{-t/\tau} \qquad \qquad \tau=(R_{L1}+R_{L2})C_F$$

The maximum voltage on the capacitor is reached when the input sine wave is at its maximum, so:

$$V_{LO} = V_{L\: max} - V_\gamma$$

The capacitor does not discharge during all the input signal period, as during part of the period the diode is **ON**. We can, however, obtain a maximum bound of the ripple considering a worse case where the discharge time is all the input signal period. The maximum ripple can be calculated then:

$$V_{L\: max} = V_{LO} \qquad \qquad V_{L\: min} = V_{LO} \cdot e^{-t/\tau}$$ 

$$\Delta V_L = V_{L\: max} - V_{L\: min} = V_{LO} \cdot \left( 1 - e^{-t/\tau} \right)$$

---

**CALCULATION TASK**  

Obtain $V_{L\: max}$, $V_{L\: min}$ and $\Delta V_L$ for our case considering the signals applied to the circuit and the values of the components.  

Compare the calculations with the measurements.

---

It is normal that the measure ripple is lower than the calculated one, as the calculation is an upper bound for the ripple.

The ripple is too high in this circuit for the output to be considered DC. In order to reduce it we need to increase the time constant $\tau$. In a power supply circuit, the load resistance in the worst case is calculated from the maximum output voltage and the maximum current the circuit shall provide. Then, the $C_F$ capacitor is calculated from a ripple specification.

In our case, as we are only learning about the circuit, we will just increase $R_L$ from $1 k\Omega$ to $10 k\Omega$ to increase 10 times the $\tau$ value.

---

**BUILD TASK**  
Change $R_L$ from $1 k\Omega$ to $10 k\Omega$

---

Now we can repeat the measurements. Remember that we don't repeat the wave generation as it is still stored inside the SLab board.

In [ ]:
# Perform the measurement and store it in some variables
t,a1,a2,a3,a4 = slab.wavePlot(returnData=True)

In [ ]:
vi2 = a3 - a2  # Calculate Vi
vo2 = a1 - a2  # Calculate Vo

# Show the Vo and Vi against time
slab.plot1n(t,[vi2,vo2],"Filtered Response for 10k","time (s)","Voltage (V)",["Vi","Vo"])

You can now compare the ripple obtained on both cases.

In fact, as is is stored in a different vector, we can use **SLab** to plot both together.

In [ ]:
# Plot both cases
slab.plot1n(t,[vo,vo2],"Ripple comparison","time (s)","Voltage (V)",["1k","10k"])

It is not really true that the diode changes from two states **ON** and **OFF**. In reality, the diode has an exponential function and conducts some current whenever its $V_D$ voltage is positive. It is apparent on the curves because the exponential **OFF** state curve is really seen only for $V_D$ voltages much lower than the $0.3 V$ value associated to $V_\gamma$.

If you detect some discrepancies between the measurements and the calculations, consider only the part of the capacitor discharge curve after the cross between the $V_i$ and $V_o$ curves.

When the ripple voltage is low, the exponential curve of the capacitor voltage for the diode in **OFF** state can be simplified using the first order Taylor approximation of the exponential.

$$V_L(t) = V_{L\: max} \cdot e^{-t/\tau} \approx V_{L\: max} \left( 1 - \frac{1}{\tau} \right)$$

In this case the ripple can be easily calculated:

$$V_{L \: min} \approx V_{L\: max} \left( 1 - \frac{1}{\tau} \right)$$

$$\Delta V_L \approx V_{L\: max} \left( 1-1 + \frac{1}{\tau} \right) = V_{L\: max} \frac{T}{\tau}$$

This is the formula usually used in power supply designs as, in this case, ripple is specified to be small.

---

**CALCULATION TASK**  
Use the above formula to obtain an approximation of the ripple we will get in the measurements.   
Compare it to the exact exponential measurement.

---



## Demodulating a signal

As said at the start of the document, obtaining a DC voltage from a sine wave is only one of the main uses of a rectifier circuit. Other of important usages of a rectifier is building amplitude demodulators. 

In order to work this subject we will first create an amplitude modulated signal (AM). The characteristics of this signal will be:

* Carrier frequency of $500 Hz$  


* Modulating signal of $10 Hz$ (1/50 of the carrier)  


* Voltage range of $\pm 2 V$ in bridge mode  


* $50%$ modulation depth   


The modulation depth parameter means that the modulating signal changes a 50% of the carrier amplitude. In order to generate the signal we first import the **numpy** module and create a 1000 points array. The modulated low frequency signal will have this number of data points.

Then we define the angular frequencies relative to this array. As the modulating signal has 1000 points, its period must be 1000 points of X. As the carrier has a frequency 50 times the modulating signal, its period should be 20 points of X.

Then we can compose the modulated signal multiplying the carrier with a signal that goes from $0.5$ to $1.0$ at the modulating signal frequency. After generating the signal we will show it on screen.

In [ ]:
import numpy as np    # Import the numpy module
  
x=np.arange(0,1000)   # Create a 1000 points linear range
wc=2*np.pi/20         # Carrier amgular frequency
wm=2*np.pi/1000       # Moldulating angular frequency

s=np.sin(wc*x)*(0.75+0.25*np.sin(wm*x))  # Generate the signal
slab.plot11(x,s,'','Sample','Value')     # Show it

The signal to send to the board shall take into account the virtual ground and the $1.2V$ maximum amplitude.

In [ ]:
v1 = 1.6 + 1.2*s                                # Compute voltage
slab.plot11(x,v1,'','Sample','Voltage[V]')      # Show it

Afterwards we will send those signals to the hardware board and set the frequency. As the full table is one cycle of the modulating signal, its frequency will be $10 Hz$.  

In [ ]:
slab.loadWavetable(v1)               # Load DAC1 signal
slab.setWaveFrequency(10)            # Set modulatingh signal frequency

In order to demodulate the signal we will use the same previous filtered rectifier circuit but we will need to change the component values to set a proper $\tau$ value. Basically we want the output of the rectifier to follow the $10 Hz$ signal envelope. If $\tau$ is too low the output will follow the full signal, not the envelope. If $\tau$ is too high, we will only get the maximum of the signal.

The maximum slope of the $10 Hz$ modulating signal is:

$$\left. \frac{\partial V_m}{\partial t} \right|_{max}=0.5V \cdot \omega _m$$

We can make this slope to coincide with the ripple:

$$\frac{\Delta V_{L\: max}}{T}=\frac{V_{L\: max}}{\tau}=0.5V \cdot \omega _m$$

From that we can determine $\tau$, and, if we fix $C_F$ to be $10 \mu F$, we can obtain $R_L$.  
In our case our modulating signal is a constant tone. In practice there will be a frequency range for the modulating signal and we will need to obtain a compromise value of $\tau$.

---

**CALCULATION TASK**  
Determine the limit value of $R_L$. Choose the resistors values that are below that value.

---

**BUILD TASK**  
Now we need to change the resistor value on the circuit and proceed with the demodulation measurements.  

---

After building the circuit we can make a transient measurement

In [ ]:
# Transient measurement for the demodulator

# Store 2 full modulating waves on four ADCs
slab.tranStore(2000,4)  
# Perform the measurement and store the results
t,a1,a2,a3,a4=slab.wavePlot(returnData=True)

After performing the measurements we can retrieve the $V_i$ and $V_o$ Voltages taking into account the virtual ground.

In [ ]:
vi=a3-a2  # Retrieve Vi from measurments
vo=a1-a2  # Retrieve Vo

# Plot Vi and Vo against time
slab.plot1n(t,[vi,vo],"Demodulator Response","time (s)","Voltages (V)",["Vi","Vo"])

In the above graph, the $V_o$ curve shall follow the envelope of the original modulated $V_i$ signal although its shape probably is not perfect.

In simple receiver designs, the rectifier does not bother using any capacitor. Try to remove the capacitor from the circuit an repeat the measurements.

---

**BUILD TASK**  
Remove the capacitor from the circuit  

---

Now, repeat the measurements using the following code cell:

In [ ]:
# Transient measurement for the demodulator

# Now we use waveResponse because we don't need a graph of the ADCs
t,a1,a2,a3,a4=slab.waveResponse()

vi=a3-a2  # Retrieve Vi from measurments
vo=a1-a2  # Retrieve Vo

# Plot Vi and Vo against time
slab.plot1n(t,[vi,vo],"Demodulator Response without capacitor","time (s)","Voltages (V)",["Vi","Vo"])

In a simple radio application, the receiver audio amplifier, together with the speaker, has usually an inherent bandwidth limitation, you can let the bandwidth to restore the modulating signal from the rectified $V_o$ signal.

## Adding some signal processing

We can filter the signal ourselves to see the effect of the bandwidth on the signal obtained from the rectifier when there is no capacitor.

This section deals with the FIR filtering capabilities of the **scipy.signal** Python package. You don't need to understand the theory of [FIR filters](https://en.wikipedia.org/wiki/Finite_impulse_response) to use them in this project as the exact commands to obtain the desired results will be provided.

The modulating signal we are generating has a 10 Hz frequency and 1000 data points, so the [sampling frequency](https://en.wikipedia.org/wiki/Sampling_(signal_processing)) is $10 kHz$ and the [Nyquist frequency](https://en.wikipedia.org/wiki/Nyquist_frequency) is $5 kHz$. The carrier frequency is $500 Hz$ so we can just filter all frequencies below $50 Hz$. In order to do that we can generate a 100 tap low pass filter with a $50 Hz$ cutoff frequency using the **firwin** function on the **scipy.signal** module.

Once we have the filter, we can apply it to the rectified signal we have previously obtained and show in a graph both the rectified signal and the filter output.

In [ ]:
# Import the scipy.signal package
import scipy.signal as signal

# Create a 100 tap 50Hz low pass filter
taps = signal.firwin(100,50,pass_zero=True,nyq=5000)

# Calculate how the demudulated signal is filtered
vof = signal.lfilter(taps,1.0,vo)

# Show the result
slab.plot1n(t,[vo,vof],"","time (s)","Vo,Vof (V)",["Demodulator Output","Filtered Signal"])

You can see that, after a start transient, the output of the filter extracts the shape of the modulating signal from the rectified one. Note that the signal has a better shape that the one we obtained by adding a capacitor in parallel with the load resistance.

## Full wave bridge rectifier

Up to this point we have worked with the basic single diode rectifier. The main drawback of this rectifier is that it only let pass the positive part of the input signal, so we don't get any energy or information from the negative part of the signal. The following circuit, that uses four diodes instead of one, is the **full wave bridge rectifier**. It is called **full wave** because it uses information not only from the positive part of the input wave but also from the negative part. It is called **bridge** because the output has a different reference than the input and is located, between two symmetric subcircuits.

![Full Wave Circuit](images/D_04/full_wave.png)

In order to solve this circuit we could use the same technique used previously for the basic rectifier. This time, as we have **four** diodes and each one could be in two states (**ON**, **OFF**), the number of combined hypothesis grow to $2^4=16$ possibilities. Testing all of them would be too much.

Fortunately, not all hypotheses are possible. In fact only three are possible and we can work out which they are from the circuit. Let's start seeing that all the circuit connected to $V_s$ is passive and resistive. That is, no element is able to store or generate energy. 

That means that if $V_s$ is positive, current must go out of it from the **(+)** terminal and return to the **(-)** terminal. Current going out of the $V_s$ **(+)** terminal reaches $D1$ and $D3$, but as $D1$ is blocking, it can only go to $D1$, as the current goes through $R_L$, from the positive to the negative terminal, voltage drops before reaching $D2$ and $D4$. Both diodes are in the correct orientation but current cannot go through $D3$ because that would mean going from a lower voltage to a higher voltage and no passive component can do that. That means that current returns to $V_s$ through diode $D4$.

In a similar way, if $V_s$ is negative, current exits the source from the **(-)** terminal and goes through $D2$, $R_L$ and $D3$ before returning to the source on the **(+)** terminal. As the current always enters the load on its **(+)** terminal, voltage on the load is always positive.

There is another case left when voltage in the source is not enough to turn the two diodes in the current path. In this case no diode conducts.

The following table shows all three cases:

![Full Wave Table](images/D_04/full_wave_table.png)

For each case we can set the voltage or current equation. The case will be possible for the $V_s$ range of values that make true all the four inequations for each diode state.

---

**CALCULATION TASK**  

Solve the circuit obtaining $V_L(V_S)$ for all three cases.  
Determine the $V_S$ range compatible with each case.  
Draw the $V_L(V_S)$ response for $V_S$ values between $-3 V$ and $3 V$. Consider a $0.7 V$ value for V_\gamma on the diodes.  

You should obtain a three region **"V"** shaped drawing, with a flat center region, for the $V_L(V_S)$ response.

---

Now, it is time to measure the circuit. The following figure shows the circuit with all connections.

![Full Wave Schematic](images/D_04/full_wave_sch2.png)

Observe the $100 k\Omega$ bias $R_B$ resistor. We don't need this resistor for the circuit to work. It will work ok regardless of having this resistor or not. The problem is measuring the circuit, not making it work. 

In **Case 3** (see above table), all diodes are in **OFF** state. That means that the load $R_L$ will be isolated from the **DACs**. All **ADC** measurements need to have a DC path to ground because the $V_L$ measurement is not intrinsically deferential. We measure **ADC1** and **ADC2** respect to ground and compute $V_L$ from the difference. 

The bias resistor $R_B$ provides the needed DC path to ground when all diodes are in **OFF** state. Sometimes you could get good measurements without this resistor but don't count on it.

---

**BUILD TASK**  
Build the above circuit

---

Now we can obtain the DC $V_o(V_i)$ response of the circuit

In [ ]:
# Obtain the DC Voltage I/O response of the full wave rectifier

slab.setVoltage(2,1.6)            # Set virtual ground on DAC2 (although it should already be that way)
data=slab.dcSweep(1,0.0,3.2,0.1)  # Sweep DAC1 voltage

vi = data[3] - data[4]   # Compute Vi
vo = data[1] - data[2]   # Compute Vo

# Plot Vo against Vi
slab.plot11(vi,vo,'Full Wave Rectifier DC Response','Vi[V]','Vo[V]')

You can now compare the response with the calculated one.

If you are curious, you can repeat the measurement removing the $100k\Omega$ resistor. This whay you can see why we are including it and if it is really needed.

After the DC measurements we can repeat the AC ones. We will set the two **DAC** waves between **0.1 V** and **3 V**. Then perform the measurement for 5 full waves.

In [ ]:
# Full Wave Rectifier Transient response

slab.waveSine(0.2,2.6,100)               # Sine wave on DAC1
slab.setWaveFrequency(50)                # Set frequency of wave
slab.waveSine(2.6,0.2,100,second=True)   # Phase inverted wave on DAC2
slab.tranStore(500,4)                    # Store 5 waves for DAC1 to DAC4

# Perform the measurement and store the data
t,a1,a2,a3,a4 = slab.waveResponse(dual=True)

vi = a3 - a4   # Compute Vi
vo = a1 - a2   # Compute Vo

# Show Vi and Vo against time
slab.plot1n(t,[vi,vo],"Full Wave Rectifier Response","time (s)","Voltages (V)",["Vi","Vo"])

Observe how the output features a larger drop from the input compared with the basic rectifier circuit. This is due to the fact that the current needs to get through **two diodes** instead of one.

We can also repeat the measurements when using a $C_F$ filter capacitor.

![Full Wave Filtered](images/D_04/full_filtered2.png)

---

**BUILD TASK**  
Modify the circuit to add the $C_F$ Capacitor  

---

The following code cell measures the transient response

In [ ]:
# Filtered Full Wave Rectifier Measurement

slab.waveSine(0.1,2.1,100)               # Wave on DAC1
slab.setWaveFrequency(50)                # Set frequency to 50Hz
slab.waveSine(2.1,0.1,100,second=True)   # Inverse phase on DAC2
slab.tranStore(500,4)                    # Store 5 full waves for four ADC channels

# Perform the measurement
t,a1,a2,a3,a4 = slab.waveResponse(dual=True)

vi = a3 - a4  # Compute Vi
vo = a1 - a2  # Compute Vo

# Show Vi and Vo against time
slab.plot1n(t,[vi,vo],"Full Wave Rectifier Filtered Response","time (s)","Voltages (V)",["Vi","Vo"])

The ripple should be lower than the one on the base rectifier because the capacitor lost charge is replaced every half period not each period.  
When the ripple is low enough to substitute the exponential with its Taylor approximation, it can be calculated as:

$$\Delta V_L = V_{L\: max}\frac{T}{2\tau}$$

## Last comments

In this project we have dealt with two rectifier topologies: the basic half wave rectifier and the full wave bridge rectifier. We have seen applications both for obtaining a DC voltage from an AC sine voltage and for demodulating an AM signal. Although those two applications are very important, the diodes can be used for many other applications.

Also, in this document we have dealt also with the need for a DC path to ground when we perform floating differential measurements. Most measurement instruments require a DC path from the measured node to the instrument ground reference and this fact is easy to be overlooked if you perform a differential measurement by subtracting two ground referenced measurements. 

## References

**SLab Python References**    
Those are the reference documents for the SLab Python modules. They describe the commands that can be carried out after importing each module.  
They should be available in the [Reference](http://localhost:8888/notebooks/Reference) folder.

**TinyCad**  
Circuit images on this document have been drawn using the free software TinyCad  
https://sourceforge.net/projects/tinycad/

## Document license

Copyright  ©  Vicente Jiménez (2018-2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">